In [1]:
import time
import subprocess
import win32gui
import pyautogui as pg

import speech_recognition as sr
import noisereduce as nr
import keyboard
from sentence_transformers import SentenceTransformer, util
import numpy as np
from google_trans_new import google_translator

In [2]:
video_path = r'videos/CatDocumentary.mp4'

vlc_path = r'C:\Users\adima\Desktop\projects\python\MediaControl\venv\Scripts\VLC\vlc.exe'

term_command = [vlc_path,
                '--video-on-top',
                '--no-volume-save',
                '--play-and-exit',
                '--snapshot-path=screenshots',
                '--snapshot-sequential',
                '--input-record-path=records',
                video_path]

In [3]:
# p = subprocess.Popen(term_command)

In [4]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [5]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))


def DLC_activation():
    fw = pg.getWindowsWithTitle('VLC')
    pg.click(fw[0].center)
    
    
def get_window_titles():
    ret = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            txt = win32gui.GetWindowText(hwnd)
            if txt:
                ret.append((hwnd, txt))

    win32gui.EnumWindows(winEnumHandler, None)
    return ret


def callback(recognizer, audio): 

    try:
        speech_as_text = recognizer.recognize_google(audio, language="ru-RU").lower()
        print('You said ', speech_as_text)

        speech_as_text_set = set(speech_as_text.split())
        if 'megogo' in speech_as_text or 'мегого' in speech_as_text:
            print('Got keyword')
            recognize_main()

    except sr.UnknownValueError:
        print('Could not understand audio')
        
    except sr.RequestError as e:
        print(f'No results from speech recognition service, {e}')


def recognize_main():
    DLC_activation()
    pg.press('down', presses=12, interval=0.07)
    
    print("Recognizing Main...")
    audio_data = r.listen(source, phrase_time_limit = 2)
    print('Recording ended')
    
#     pg.press('up', presses=12, interval=0.05)
    
    try:
        txt = r.recognize_google(audio_data, language="ru-RU").lower()
        print('Your command ', txt)
        control_player(txt)
        
    except sr.UnknownValueError:
        print('Could not understand main command')
        pg.press('up', presses=12, interval=0.05)
        
    except sr.RequestError as e:
        print(f'No results from speech recognition service in main command, {e}')
        pg.press('up', presses=12, interval=0.05)
    
    

def control_player(command):
    translated_command = translator.translate(command)[:-1]
    print('Translated command ', translated_command)
    
    enc_command = model.encode([translated_command])[0]
    
    
    press = True
    
     # forward
#     if cosine(model.encode(['rewind forward'])[0], enc_command) > 0.7:
#         DLC_activation()
#         print('8')
#         keyboard.send("alt+right")
        
#     # backwards
#     elif cosine(model.encode(['rewind backward'])[0], enc_command) > 0.7:
#         DLC_activation()
#         print('9')
#         keyboard.send("alt+left")
        
        
    # закрыть видео
    if cosine(model.encode(['stop'])[0], enc_command) > 0.8 or \
                    cosine(model.encode(['the end'])[0], enc_command) > 0.8 or \
                    cosine(model.encode(['close'])[0], enc_command) > 0.8:
        DLC_activation()
        print('1')
        keyboard.send("ctrl+q")
        press = False
        
    # pause and play
    elif cosine(model.encode(['pause'])[0], enc_command) > 0.8 or \
                    cosine(model.encode(['play'])[0], enc_command) > 0.8 or \
                    cosine(model.encode(['continue'])[0], enc_command) > 0.8:
        DLC_activation()
        print('2')
        keyboard.send("space")
        
    #screenshot
    elif cosine(model.encode(['take screenshot'])[0], enc_command) > 0.7 or \
                    cosine(model.encode(['take a picture'])[0], enc_command) > 0.7:
        DLC_activation()
        print('4')
        keyboard.send("shift+s")
        
    # start and stop recording video
    elif cosine(model.encode(['start screen recording'])[0], enc_command) > 0.7 or \
                    cosine(model.encode(['finish screen recording'])[0], enc_command) > 0.7:
        DLC_activation()
        print('5')
        keyboard.send("shift+r")
    
    
     # increase volume
    elif cosine(model.encode(['increase volume'])[0], enc_command) > 0.7:
        DLC_activation()
        print('6')
        pg.press('up', presses=4, interval=0.01)
        
#     # decrease volume
#     elif cosine(model.encode(['decrease reduce volume'])[0], enc_command) > 0.7:
#         DLC_activation()
#         print('7')
#         pg.press('down', presses=4, interval=0.01)
        
    pg.press('up', presses=12, interval=0.07) if press==True else None
    

def initialize_player():
    p = subprocess.Popen(term_command)
    time.sleep(1.5)
    DLC_activation()

    tempWindowName = win32gui.GetWindowText(win32gui.GetForegroundWindow())

    all_titles = get_window_titles()
    window_starts = lambda title: [(hwnd, full_title) for (hwnd, full_title) in all_titles if full_title.startswith(title)]
    all_matching_windows = window_starts(tempWindowName)
    print('Window', all_matching_windows[0])

    return all_matching_windows[0][0]


def check_player(matching_window):
    play = True
    while play:
        try:
            if win32gui.IsWindow(matching_window):
                pass
            else:
                raise Exception
        except Exception as e:
            print(f'Programm stopped. Exception: {e} \n Break')
            stop_listening()
            print('Recognition stopped')
            break
        time.sleep(0.5)
        
        


In [33]:
r = sr.Recognizer()
m = sr.Microphone()
translator = google_translator()  

with m as source:
    r.adjust_for_ambient_noise(source)
    
    

stop_listening = r.listen_in_background(source, callback, phrase_time_limit=2)
matching_window = initialize_player()
check_player(matching_window)


Window (1378882, 'videos/CatDocumentary.mp4 - Медиапроигрыватель VLC')
Could not understand audio
You said  megogo
Got keyword
Recognizing Main...
Recording ended
Your command  поставь на паузу
Translated command  Put on the pause
2
You said  megogo
Got keyword
Recognizing Main...
Recording ended
Your command  продолжить
Translated command  proceed
2
Could not understand audio
You said  megogo
Got keyword
Recognizing Main...
Recording ended
Your command  сделай скриншот
Translated command  Make a screenshot
4
You said  megogo
Got keyword
Recognizing Main...
Recording ended
Your command  сделай громче
Translated command  Make it louder
6
You said  megogo
Got keyword
Recognizing Main...
Recording ended
Your command  закрой видео
Translated command  Close video
1
Programm stopped. Exception:  
 Break
Recognition stopped
